# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port lincoln,-34.7333,135.8667,12.52,87,100,2.16,AU,1718661558
1,1,west island,-12.1568,96.8225,24.99,94,75,4.63,CC,1718661558
2,2,puerto natales,-51.7236,-72.4875,-4.75,86,75,1.54,CL,1718661559
3,3,st. john's,47.5649,-52.7093,16.82,64,20,6.17,CA,1718661347
4,4,blackmans bay,-43.0167,147.3167,5.96,94,100,1.71,AU,1718661559


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_city=city_data_df.hvplot.points("Lng","Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    size = "Humidity",
    color = "City"  
)
# YOUR CODE HERE

# Display the map
map_plot_city
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
results_city=city_data_df[(city_data_df['Humidity'] < 70) & 
                                  (city_data_df['Cloudiness'] == 0) & 
                                  (city_data_df['Max Temp'] <= 27) & 
                                  (city_data_df['Max Temp'] >= 21)]
# YOUR CODE HERE

# Drop any rows with null values
results_city_null_values= results_city.dropna()
# YOUR CODE HERE

# Display sample data
results_city_null_values
# YOUR CODE HERE

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,ipora,-16.4419,-51.1178,21.54,34,0,0.85,BR,1718661561
48,48,santa maria da vitoria,-13.3947,-44.1886,22.67,38,0,0.98,BR,1718661568
62,62,miracema do tocantins,-9.5618,-48.3967,26.79,39,0,1.55,BR,1718661570
108,108,kill devil hills,36.0307,-75.6760,26.10,58,0,4.63,US,1718661476
141,141,tinghir,31.5157,-5.5343,27.00,11,0,3.05,MA,1718661585
145,145,al bawiti,28.3492,28.8659,26.49,35,0,4.33,EG,1718661585
158,158,barra do bugres,-15.0725,-57.1811,26.72,36,0,1.27,BR,1718661588
189,189,xique xique,-10.8217,-42.7311,25.06,50,0,1.92,BR,1718661593
206,206,san michele salentino,40.6316,17.6315,25.67,60,0,2.69,IT,1718661596
216,216,tadine,-21.5500,167.8833,23.87,65,0,4.45,NC,1718661598


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=results_city[['City', 'Lat', 'Lng', 'Humidity']].copy()
# YOUR CODE HERE 'Country'

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''
# YOUR CODE HERE

# Display sample data
hotel_df
# YOUR CODE HERE

,City,Lat,Lng,Humidity,Hotel Name
15,ipora,-16.4419,-51.1178,34,
48,santa maria da vitoria,-13.3947,-44.1886,38,
62,miracema do tocantins,-9.5618,-48.3967,39,
108,kill devil hills,36.0307,-75.6760,58,
141,tinghir,31.5157,-5.5343,11,
145,al bawiti,28.3492,28.8659,35,
158,barra do bugres,-15.0725,-57.1811,36,
189,xique xique,-10.8217,-42.7311,50,
206,san michele salentino,40.6316,17.6315,60,
216,tadine,-21.5500,167.8833,65,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 1000# YOUR CODE HERE
params = {"apiKey": geoapify_key,
    "limit":20,
    'categories':'accommodation.hotel'
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"# YOUR CODE HERE
    params["bias"] = f"proximity:{longitude},{latitude}"# YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)# YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = name_address.json()# YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ipora - nearest hotel: No hotel found
santa maria da vitoria - nearest hotel: No hotel found
miracema do tocantins - nearest hotel: No hotel found
kill devil hills - nearest hotel: Mariner Days Inn & Suites
tinghir - nearest hotel: Riad Agraw
al bawiti - nearest hotel: Old Oasis Hotel
barra do bugres - nearest hotel: Hotel American Palace
xique xique - nearest hotel: Hotel J. Macedo
san michele salentino - nearest hotel: No hotel found
tadine - nearest hotel: No hotel found
urla - nearest hotel: No hotel found
pacific grove - nearest hotel: Pacific Grove Inn
mikhaylovsk - nearest hotel: No hotel found
talara - nearest hotel: Hospedaje El Aypate
alvorada - nearest hotel: No hotel found
carloforte - nearest hotel: Hotel Riviera
pimentel - nearest hotel: Hospedaje Calle del Sol


,City,Lat,Lng,Humidity,Hotel Name
15,ipora,-16.4419,-51.1178,34,No hotel found
48,santa maria da vitoria,-13.3947,-44.1886,38,No hotel found
62,miracema do tocantins,-9.5618,-48.3967,39,No hotel found
108,kill devil hills,36.0307,-75.6760,58,Mariner Days Inn & Suites
141,tinghir,31.5157,-5.5343,11,Riad Agraw
145,al bawiti,28.3492,28.8659,35,Old Oasis Hotel
158,barra do bugres,-15.0725,-57.1811,36,Hotel American Palace
189,xique xique,-10.8217,-42.7311,50,Hotel J. Macedo
206,san michele salentino,40.6316,17.6315,60,No hotel found
216,tadine,-21.5500,167.8833,65,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display 

# Configure the map plot 
map_plot_city2 = hotel_df.hvplot.points("Lng","Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height =600,
    size = "Humidity",
    color = "City",
    hover_cols = ['City', 'Country', 'Hotel Name'] 
)
# YOUR CODE HERE

# Display the map
map_plot_city2
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)